# Cleaning Dataset

import necessary modules and load dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
welcome = pd.read_csv("../../../Data & Script/welcome_apc.csv", encoding = "ISO-8859-1")

overview of the dataset

In [2]:
print(welcome.head(5))
print("length of dataset = ", len(welcome))

              PMID/PMCID Publisher           Journal title  \
0                    NaN       CUP  Psychological Medicine   
1             PMC3679557       ACS       Biomacromolecules   
2  23043264  PMC3506128        ACS              J Med Chem   
3    23438330 PMC3646402       ACS              J Med Chem   
4   23438216 PMC3601604        ACS              J Org Chem   

                                       Article title  \
0  Reduced parahippocampal cortical thickness in ...   
1  Structural characterization of a Model Gram-ne...   
2  Fumaroylamino-4,5-epoxymorphinans and related ...   
3  Orvinols with mixed kappa/mu opioid receptor a...   
4  Regioselective opening of myo-inositol orthoes...   

  COST (£) charged to Wellcome (inc VAT when charged)  
0                                              £0.00   
1                                           £2381.04   
2                                            £642.56   
3                                            £669.64   
4         

The following function takes a column name or attribute and returns duplicate values

In [3]:
import collections

def show_duplicates(attribute):
    mydict = dict(collections.Counter(welcome[attribute]))
    duplicate_keys = []
    for key in mydict:
        if(mydict[key] != 1):
            duplicate_keys.append(key)
    return mydict, duplicate_keys

Find articles repeated

In [4]:
mydict, duplicate_keys = show_duplicates('Article title')
print(duplicate_keys)

['Exclusive breastfeeding, diarrhoel morbidity and all-couse mortality in infnats of HIV infected']


find duplicate PMID/PMCID

In [5]:
mydict, duplicate_keys = show_duplicates('PMID/PMCID')
print(duplicate_keys)

[nan, 'PMC in progress', 'PMC3173209', 'Monograph chapter, not in PMC', 'PMC 3599138', 'In Process', 'pub Aug 2013', 'Epub ahead of print pub Feb 2013, print in press', 'Not yet available', 'e-pub ', 'print in press', 'Epub ahead of print April 2013 - print in press', '-', 'Pub July 2013', 'PMC3746156', 'not yet published', '41609', 'Epub ahead of print - june 2013', '22735079', 'Epub ahead of print pub Jan 2013, print in press', 'PMC 3413714', 'PMC 3435256', 'Pending', 'PMC3708033\n\n', 'PMCID:\n    PMC3647051\n', 'PMC3597274', 'PMC3405234', 'PMC3529206']


In the next cells I will clean the different formats used for PMCID and PMID. The issue of duplicate id cannot be resolved from a data cleaning perspective because it may have occured because of some errors and unless there is a doman expert it is impossible to determine the real values.

In an attempt to make a uniform format for ID I tried filtering the numbers(or digits) out of each string. In this way it is possible to get a list of numbers. The PMIDs are 8 digits and PMCIDs are 7 digits. So it is possible to analyse the lists and create a uniform PMID/PMCID format for all. 

In [6]:
welcome['PMID/PMCID'].apply(lambda x: list(filter(str.isdigit, str(x)))).head()

0                                               []
1                            [3, 6, 7, 9, 5, 5, 7]
2    [2, 3, 0, 4, 3, 2, 6, 4, 3, 5, 0, 6, 1, 2, 8]
3    [2, 3, 4, 3, 8, 3, 3, 0, 3, 6, 4, 6, 4, 0, 2]
4    [2, 3, 4, 3, 8, 2, 1, 6, 3, 6, 0, 1, 6, 0, 4]
Name: PMID/PMCID, dtype: object

I prefered to use a regular expression to find all numbers whose number of digits is greater than seven. Those are assumed to be either PMID or PMCID numbers. The regex is applied for each row and it returns a list of numbers, function mix takes a list and mixes it correclty, in the desired format

In [7]:
def mix(row):
    # if length is one, it is a single number
    if(len(row) == 1):
        return row[0]
    # if length is two
    elif(len(row) == 2):
        # if order of numbers PMID, PMCID, then join them into one string
        if(len(row[0]) == 8 and len(row[1]) == 7):
            return '/'.join(row)
        # in the reverse order, so rearrange and join
        else:
           row[0], row[1] = row[1], row[0]
           return "/".join(row)
    # make the rest NA
    else:
        return "NA"

welcome['PMID/PMCID'] = welcome['PMID/PMCID'].apply(lambda x: mix(re.findall('\d{7,}', str(x))))

Finding the number of unique IDs

In [8]:
print(len(welcome['Journal title'].unique()))
groups = welcome.groupby('Journal title').groups
print(len(groups.keys()))

985
984


There are 985 or 984 keys let me see after converting them into lower case and strip white space. Hopefully, it removes some redundancies in the data.

In [9]:
welcome['Journal title'] = welcome['Journal title'].str.lower().str.strip()

In [10]:
print(len(welcome['Journal title'].unique()))
groups = welcome.groupby('Journal title').groups
print(len(groups.keys()))

895
894


Test some regular expression to find all rows which start with j and followed by med and then chem all separated by other characters including space

In [11]:
pattern = re.compile(r'j.*med\w*\schem\w*')
welcome['Journal title'].apply(lambda x: re.findall(pattern, str(x)))

0                                     []
1                                     []
2                           [j med chem]
3                           [j med chem]
4                                     []
5       [journal of medicinal chemistry]
6                                     []
7                                     []
8                                     []
9                                     []
10                                    []
11      [journal of medicinal chemistry]
12      [journal of medicinal chemistry]
13                                    []
14      [journal of medicinal chemistry]
15                                    []
16                                    []
17                                    []
18                                    []
19                                    []
20                                    []
21                                    []
22                                    []
23                                    []
24              

In an effort to clean Journal title the following regular expressons are applied.

1. find those starting with j and replace them with journal of
2. replace chem or chem. with chemistry
3. replace med medicine
4. bio or biol with biological
5. org with organic
6. change the journal into journal


In [12]:
pattern1 = re.compile(r'^j\b')

#print(welcome['Journal title'].apply(lambda x: re.findall(pattern, str(x))))

welcome['Journal title'] = welcome['Journal title'].apply(lambda x: re.sub(pattern1, 'journal of',str(x)))

pattern2 = re.compile(r'\bchem\.?$')

#print(welcome['Journal title'].apply(lambda x: re.findall(pattern2, str(x))).head(70))

welcome['Journal title'] = welcome['Journal title'].apply(lambda x: re.sub(pattern2, 'chemistry',str(x)))

pattern3 = re.compile(r'\bmed\b')

#print(welcome['Journal title'].apply(lambda x: re.findall(pattern3, str(x))).iloc[55:])

welcome['Journal title'] = welcome['Journal title'].apply(lambda x: re.sub(pattern3, 'medicinal',str(x)))

pattern4 = re.compile(r'\b(biol)\b')

welcome['Journal title'] = welcome['Journal title'].apply(lambda x: re.sub(pattern4, 'biological',str(x)))

pattern5 = re.compile(r'\borg\b')

welcome['Journal title'] = welcome['Journal title'].apply(lambda x: re.sub(pattern5, 'organic',str(x)))

pattern6 = re.compile(r"(the)\s(journal)\b")

#print(welcome['Journal title'].apply(lambda x: re.findall(pattern6, str(x))).iloc[1850:])

# I made two groups and removed the first group for all cells in Journal title column
welcome['Journal title'] = welcome['Journal title'].apply(lambda x: pattern6.sub(r'\2', str(x)))

# unique values
#welcome['Journal title'].apply(lambda x: ''.join(re.findall(pattern1, str(x)))).value_counts()
#print(welcome['Journal title'].apply(lambda x: ''.join(re.findall(pattern1, str(x)))).value_counts())
#print(welcome['Journal title'].apply(lambda x: re.findall(pattern2, str(x))).iloc[0:])

 Fixing acta d crystallographica
 
- acta crystallographica section d: biological crystallography
- acta crystallographica, section d
- acta crystallography d
- acta d
- acta crystallographica section d,  biological crystallography
- acta crystallographica section f: structural biology and crystallization communications
- acta crystallographica section f: structural biology and crystallization communications

Fixing acta f crystallographica

- acta f
- acta crystallographica section f: structural biology and crystallization communications
- acta crystallographica section f: structural biology and crystallization communications

Fixing acta neuro

- acta neuropathol
- acta neuropathologica

Replace all & by and


In [13]:
# Note: if you remove ? it only match strings where is preceded by space and d is optionally followed by either colon or comma
pattern7 = re.compile(r'^(acta)(.*\sd)([:,]?)(.*)')

#print(welcome['Journal title'].apply(lambda x: re.findall(pattern7, str(x))).loc[920:])

welcome['Journal title'] = welcome['Journal title'].apply(lambda x: re.sub(pattern7, 'acta crystallographica section d: biological crystallography',str(x)))

# Note: if you remove ? it only match strings where is preceded by space and d is optionally followed by either colon or comma
pattern8 = re.compile(r'^(acta)(.*\sf)([:,]?)(.*)')

#print(welcome['Journal title'].apply(lambda x: re.findall(pattern8, str(x))).loc[920:])

welcome['Journal title'] = welcome['Journal title'].apply(lambda x: re.sub(pattern8, 'acta crystallographica section f: structural biology and crystallization communications',str(x)))


pattern9 = re.compile(r'(acta neuropathol)(.*)')

#print(welcome['Journal title'].apply(lambda x: re.findall(pattern9, str(x))).loc[1717:])

welcome['Journal title'] = welcome['Journal title'].apply(lambda x: re.sub(pattern9, 'acta neuropathologica',str(x)))

# replace & by and
welcome['Journal title'] = welcome['Journal title'].str.replace('&', 'and')

In [14]:
welcome['Journal title'].loc[153:]

153               mcp (molecular and cellular proteomics)
154                     molecular and cellular proteomics
155                       journal of biological chemistry
156                       journal of biological chemistry
157                       journal of biological chemistry
158                       journal of biological chemistry
159                       journal of biological chemistry
160                       journal of biological chemistry
161                              jnl biological chemistry
162                          journal biological chemistry
163                       journal of biological chemistry
164                       journal of biological chemistry
165                                infection and immunity
166                                   journal of virology
167                 antimicrobial agents and chemotherapy
168                                     journal of vision
169                 current topics in medicinal chemistry
170           

In [15]:
 
result = welcome.groupby(['Journal title']).agg(['count'])['Article title', ]

result.sort_values(by = ['count'], ascending = False).head(20)


,count
Journal title,
plos one,190
journal of biological chemistry,67
neuroimage,29
nucleic acids research,26
plos pathogens,24
plos genetics,24
proceedings of the national academy of sciences,22
plos neglected tropical diseases,20
nature communications,19


Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal 

In [16]:
print("before rename", welcome.columns)
# rename column
welcome = welcome.rename(index=str, columns={"COST (£) charged to Wellcome (inc VAT when charged)": "Cost"})
print("after rename: ", welcome.columns)

before rename Index(['PMID/PMCID', 'Publisher', 'Journal title', 'Article title',
       'COST (£) charged to Wellcome (inc VAT when charged)'],
      dtype='object')
after rename:  Index(['PMID/PMCID', 'Publisher', 'Journal title', 'Article title', 'Cost'], dtype='object')


In [17]:
# remove currency sign
welcome['Cost']= welcome['Cost'].str.replace('£', '')
welcome['Cost']= welcome['Cost'].str.replace('$', '')

# convert to float
welcome['Cost'] = welcome['Cost'].apply(float)

In [18]:
#one way of doing
#print(welcome.groupby('Journal title')['Cost'].mean())

In [19]:
welcome.groupby(['Journal title'])['Cost'].agg(['count', 'mean','median','std']).fillna(0)


,count,mean,median,std
Journal title,,,,
academy of nutrition and dietetics,1,2379.540000,2379.540,0.000000
acs chemical biology,5,1418.186000,1294.590,507.309560
acs chemical neuroscience,1,1186.800000,1186.800,0.000000
acs nano,2,668.140000,668.140,35.708892
acta crystallographica section d: biological crystallography,7,973.847143,771.420,602.275443
acta crystallographica section f: structural biology and crystallization communications,3,782.723333,785.600,26.502351
acta neuropathologica,4,2096.057500,2076.005,238.449669
acta opthalmologica,1,2270.160000,2270.160,0.000000
acta physiol,1,1991.500000,1991.500,0.000000


In [20]:
welcome.to_csv("testxx.csv")